In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,2398
2,Huelva,Confirmados PDIA 14 días,733
3,Huelva,Tasa PDIA 14 días,"143,51640649015258"
4,Huelva,Confirmados PDIA 7 días,382
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,8
629,Municipio de Huelva sin especificar,Total Confirmados,41
630,Municipio de Huelva sin especificar,Curados,10


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  2398.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  628.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1528 personas en los últimos 7 días 

Un positivo PCR cada 639 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,2398.0,382.0,733.0,510743.0,74.792998,143.516406,213.0
Huelva-Costa,1247.0,205.0,413.0,288115.0,71.152144,143.345539,105.0
Condado-Campiña,791.0,106.0,207.0,155057.0,68.361957,133.499294,62.0
Huelva (capital),628.0,94.0,225.0,143663.0,65.430904,156.616526,45.0
Sierra de Huelva-Andévalo Central,319.0,63.0,96.0,67571.0,93.235264,142.072783,37.0
Lepe,186.0,41.0,64.0,27431.0,149.465933,233.312675,17.0
Cortegana,39.0,26.0,29.0,4666.0,557.222460,621.517360,17.0
Almonte,90.0,24.0,34.0,24191.0,99.210450,140.548138,14.0
Aljaraque,98.0,14.0,25.0,21260.0,65.851364,117.591722,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),26.0,14.0,24.0,516.0,2713.178295,4651.162791,9.0
Villarrasa,30.0,3.0,19.0,2176.0,137.867647,873.161765,1.0
Cortegana,39.0,26.0,29.0,4666.0,557.222460,621.517360,17.0
Cerro de Andévalo (El),18.0,13.0,13.0,2364.0,549.915398,549.915398,4.0
Villanueva de los Castillejos,25.0,7.0,15.0,2820.0,248.226950,531.914894,5.0
Marines (Los),2.0,2.0,2.0,396.0,505.050505,505.050505,1.0
Valdelarco,3.0,1.0,1.0,239.0,418.410042,418.410042,NaN
Jabugo,15.0,1.0,9.0,2250.0,44.444444,400.000000,NaN
Puerto Moral,1.0,1.0,1.0,282.0,354.609929,354.609929,NaN
